# Women's basketball analysis 

### Reading in data ###

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_excel("WBB.xlsx").dropna(axis='index', how='all')
df = df.reset_index(drop=True)

df

### Subset with only game (Q1-Q4) data ###

In [ ]:
game_df = df[df['Types'].isin(['Q1', 'Q2', 'Q3', 'Q4'])]
game_df = game_df.reset_index(drop=True)
game_df.head(50)

### Add **Game Total** row to `game_df` ###

In [ ]:
dates = game_df['Phase begin date (UTC)'].unique()
names = game_df['Name'].unique()


for date in dates:
    AALs = []
    for name in names:
        AAL_sum = game_df.loc[(game_df['Phase begin date (UTC)'] == date) & (game_df['Name'] == name), 'Accumulated Acceleration Load'].replace(0, np.nan).sum()
        AAL_max = game_df.loc[(game_df['Phase begin date (UTC)'] == date) & (game_df['Name'] == name), 'Accumulated Acceleration Load'].replace(0, np.nan).max()
        AAL_min = game_df.loc[(game_df['Phase begin date (UTC)'] == date) & (game_df['Name'] == name), 'Accumulated Acceleration Load'].replace(0, np.nan).min()
        if AAL_sum >= 0.00000001:
            AALs.append(AAL_sum)
        row = ({
            'Name': name,
            'Types': 'Game',
            'Phase begin date (UTC)': date,
            'Phase end date (UTC)': date,
            'Game Total': AAL_sum,
            'Game Max': AAL_max,
            'Game Min': AAL_min
        })
        df2 = pd.DataFrame(row, index=[len(game_df)])
        game_df = pd.concat([game_df, df2])
    row = ({
        'Name': 'Team',
        'Types': 'Game',
        'Phase begin date (UTC)': date,
        'Phase end date (UTC)': date,
        'Game Total': sum(AALs) / len(AALs)
    })
    df2 = pd.DataFrame(row, index=[len(game_df)])
    game_df = pd.concat([game_df, df2])
game_df = game_df.sort_values(['Phase begin date (UTC)', 'Phase ID']).drop_duplicates()
game_df.tail(60)
    
    

### Plot most recent game ###

In [ ]:
recent = game_df.loc[(game_df['Phase begin date (UTC)'] == dates[-1]) & (game_df['Types'] == 'Game'), ['Name', 'Game Total', 'Game Max', 'Game Min']].round()
recent = recent.set_index('Name')

sns.set_style('darkgrid')
fig = recent.plot.bar(color={'Game Total': '#7BAFD4', 'Game Max': '#039C07', 'Game Min': '#DC392F'}, grid=True, width=.7, linewidth=1)

plt.title(str(dates[-1])[0:10])
plt.xlabel('')
plt.ylabel('Accumulated Acceleration Load (Q1-Q4)')
plt.yticks(range(0, int(recent['Game Total'].max()) + 100, 100))

for i in range(len(fig.containers)):
    fig.bar_label(fig.containers[i])


### Add  **season total** row to `game_df` ###

In [ ]:
names = list(names)
names.append('Team')
for name in names:
    season_avg = game_df.loc[game_df['Name'] == name, 'Game Total'].mean()
    row = ({
        'Name': name,
        'Types': 'Season',
        'Season Average': season_avg
    })
    df2 = pd.DataFrame(row, index=[len(game_df)])
    game_df = pd.concat([game_df, df2])
game_df = game_df.sort_values(['Phase begin date (UTC)', 'Phase ID']).drop_duplicates()

game_df.tail(60)

### Plot season averages ###

In [ ]:
season = game_df.copy()

if 'Team' in names:
    names.pop()

season = game_df.loc[game_df['Types'] == 'Season', ['Name', 'Season Average']]

for name in names:
    season.loc[season['Name'] == name, 'Highest Game'] = game_df.loc[(game_df['Name'] == name) & (game_df['Types'] == 'Game'), 'Game Total'].replace(0, np.nan).max()
    season.loc[season['Name'] == name, 'Lowest Game'] = game_df.loc[(game_df['Name'] == name) & (game_df['Types'] == 'Game'), 'Game Total'].replace(0, np.nan).min()
season = season.set_index('Name').round()    

sns.set_style('darkgrid')
fig = season.plot.bar(color={'Season Average': '#7BAFD4', 'Highest Game': '#039C07', 'Lowest Game': '#DC392F'}, grid=True, width=.7, linewidth=1)

today = np.datetime64('today', 'D')

plt.title(f'Season (created {today})')
plt.xlabel('')
plt.ylabel('Accumulated Acceleration Load')
plt.yticks(range(0, int(season['Highest Game'].max()) + 100, 100))

for i in range(len(fig.containers)):
    fig.bar_label(fig.containers[i])

In [ ]:
%load_ext watermark
%watermark -v -m -p pandas,numpy,seaborn,matplotlib,openpyxl 